In [2]:
import sqlite3
import spacy
import re
from datetime import datetime
import time, random, sys


In [ ]:
# --- CONFIGURATION ---
DB_FILE = r"D:\DB Browser for SQLite\chatbot_db.db" # Updated to your file path
# Load the small English model for SpaCy.
# You need to download it first by running: python -m spacy download en_core_web_sm
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Spacy model 'en_core_web_sm' not found.")
    print("Please run: !python -m spacy download en_core_web_sm in a Jupyter cell or")
    print("python -m spacy download en_core_web_sm in your terminal.")
    exit()

# --- 1. DATABASE SETUP ---

def setup_database():
    """
    Connects to the existing database to ensure it is accessible and contains the required tables.
    This function no longer creates tables or populates data.
    """
    try:
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()

        # Check if the required tables exist to provide a helpful error message.
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='faq_db'")
        if cursor.fetchone() is None:
            print(f"Error: Table 'faq_db' not found in {DB_FILE}.")
            print("Please ensure your database has the correct table schema.")
            exit()
        
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='faq_db_pattern'")
        if cursor.fetchone() is None:
            print(f"Error: Table 'faq_db_pattern' not found in {DB_FILE}.")
            print("Please ensure your database has the correct table schema.")
            exit()
            
        conn.close()
        print("Database connection successful. Using existing data.")
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        exit()


# --- 2. NLP & INTENT RECOGNITION ---

def preprocess_text(text):
    """
    Processes user input by converting to lowercase, removing punctuation,
    and lemmatizing tokens.
    
    Args:
        text (str): The raw user input.
        
    Returns:
        list: A list of lemmatized tokens.
    """
    doc = nlp(text.lower())
    lemmas = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]
    return lemmas

def get_intent(user_input):
    """
    Determines the user's intent by matching their preprocessed input against
    patterns from the database. It calculates a score for each potential intent
    and returns the one with the highest score.

    Args:
        user_input (str): The raw text from the user.

    Returns:
        tuple: A tuple containing the best intent (str) and any extracted entity (like an order number).
    """

    lemmas = preprocess_text(user_input)
    lemmatized_input_str = " ".join(lemmas)

    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("SELECT intent, type, pattern, weight FROM faq_db_pattern")
    all_patterns = cursor.fetchall()
    conn.close()

    intent_scores = {}
    extracted_entity = None

    for intent, type, pattern, weight in all_patterns:
        if intent not in intent_scores:
            intent_scores[intent] = 0.0

        if type == 'keyword':
            # For keywords, we also lemmatize the pattern for a fair comparison
            pattern_lemmas = preprocess_text(pattern)
            # Check if all words in the pattern are in the user's input
            if all(p_lemma in lemmas for p_lemma in pattern_lemmas):
                intent_scores[intent] += weight
                # print(f"[MATCH] intent={intent}, type=keyword, pattern='{pattern}', +{weight} → total={intent_scores[intent]}")
        
        elif type == 'regex':
            # For regex, we match against the original (lowercased) input
            match = re.search(pattern, user_input.lower())
            if match:
                intent_scores[intent] += weight
                # print(f"[MATCH] intent={intent}, type=regex, pattern='{pattern}', +{weight} → total={intent_scores[intent]}")
                # If the regex has a capturing group, we extract it as an entity
                if match.groups():
                    extracted_entity = match.group(1)
                    # print(f"   [ENTITY] extracted → {extracted_entity}")
    
    # Show all final scores before picking
    # print("\n--- Final intent scores ---")
    # for intent, score in intent_scores.items():
    #     print(f"{intent}: {score}")


    # Determine the best intent
    if not any(score > 0 for score in intent_scores.values()):
        best_intent = 'fallback'
    else:
        best_intent = max(intent_scores, key=intent_scores.get)

    return best_intent, extracted_entity


def get_answer_for_intent(intent):
    """
    Retrieves the corresponding answer for a given intent from the database.

    Args:
        intent (str): The intent to find an answer for.

    Returns:
        str: The answer text.
    """
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("SELECT answer FROM faq_db WHERE intent = ?", (intent,))
    result = cursor.fetchone()
    conn.close()
    
    # It's good practice to have a default fallback answer in your database
    # but this handles cases where an intent might not have a matching answer.
    if result is None:
        return get_answer_for_intent('fallback')
    return result[0]

# --- 3. DECISION TREE (CONVERSATIONAL FLOW) & MAIN LOOP ---

def chatbot_response(user_input, conversation_context):
    """
    This function acts as the main decision-making hub for the chatbot.
    It gets the intent and decides the response, handling simple Q&A
    as well as multi-turn interactions.

    Args:
        user_input (str): The user's most recent message.
        conversation_context (dict): A dictionary to maintain state across turns.

    Returns:
        tuple: A tuple containing the bot's response (str) and the updated context (dict).
    """
    # If the bot is waiting for specific information (e.g., an order number)
    if conversation_context.get('waiting_for') == 'order_number':
        order_number = re.search(r'\d+', user_input)
        if order_number:
            response = f"Thank you! I am now checking the status for order #{order_number.group(0)}. Please give me a moment."
            # Reset the context
            conversation_context = {}
        else:
            response = "That doesn't look like a valid order number. Please provide the numeric order ID."
        return response, conversation_context

    # Standard intent recognition flow
    intent, entity = get_intent(user_input)
    
    # --- This IF/ELIF block is the "Decision Tree" ---
    if intent == 'order_status':
        if entity: # An order number was extracted directly by the regex
            response = f"Thank you! I am now checking the status for order #{entity}. Please give me a moment."
            conversation_context = {}
        else: # No order number found, ask for it
            response = get_answer_for_intent(intent)
            conversation_context['waiting_for'] = 'order_number'
    
    elif intent == 'goodbye':
        response = get_answer_for_intent(intent)
        # Special case to end the conversation
        conversation_context['end_session'] = True

    else: # For all other simple intents
        response = get_answer_for_intent(intent)
        conversation_context = {} # Reset context after a simple answer

    return response, conversation_context


def main():
    # One-line, anchored regex so we only trigger when the whole input is an end cue.
    END_TRIGGER_RE = re.compile(
        r"^\s*(?:"
        r"quit|exit|bye|goodbye|"
        r"ok(?:ay)?|can|done|got it|"
        r"no more|nothing else|that(?:'|’)?s all|all good|i(?:'|’)?m good|im good|"
        r"no thanks|no thank you"
        r")\s*[.!?]*\s*$",
        re.IGNORECASE
    )
    
    """Main function to run the chatbot."""
    # Ensure the database is ready and accessible
    setup_database()
    
    print("\n--- E-commerce Chatbot ---")
    print("Bot: Hello! How can I assist you? (Type 'quit' to exit)")

    conversation_context = {}
    
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            print("Bot: Goodbye!")
            break
        
        response, conversation_context = chatbot_response(user_input, conversation_context)
        print(f"Bot: {response}")

        if conversation_context.get('end_session'):
            break

if __name__ == "__main__":
    main()

Database connection successful. Using existing data.

--- E-commerce Chatbot ---
Bot: Hello! How can I assist you? (Type 'quit' to exit)


KeyboardInterrupt: Interrupted by user

New -- adding get user details

In [20]:
EMAIL_REGEX = r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$"

# --- CONFIGURATION ---
DB_FILE = r"D:\DB Browser for SQLite\chatbot_db.db" # Updated to your file path
# Load the small English model for SpaCy.
# You need to download it first by running: python -m spacy download en_core_web_sm
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Spacy model 'en_core_web_sm' not found.")
    print("Please run: !python -m spacy download en_core_web_sm in a Jupyter cell or")
    print("python -m spacy download en_core_web_sm in your terminal.")
    exit()


def ensure_user_table():
    """
    Creates user_profile table if it doesn't exist.
    """
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS user_profile (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT NOT NULL UNIQUE,
            created_at TEXT NOT NULL
        )
    """)
    conn.commit()
    conn.close()

def capture_user_profile():
    """
    Ask for name and email in a single prompt like:
    'Jane Doe jane@example.com'
    Extracts the email via regex; uses the remaining text as name.
    If the email exists in DB, keeps the stored name.
    """
    while True:
        raw = input("Hi! please give me your name and email: ").strip()
        m = re.search(EMAIL_REGEX, raw)
        if not m:
            print("I couldn't find a valid email. Try again (e.g., Jane Doe jane@example.com).")
            continue

        email = m.group(0)
        # Remove the email token from the input to get the name
        name = raw.replace(email, "").strip(" ,;<>\"'")

        if not name:
            name = input("Got your email. What's your name? ").strip()
            if not name:
                print("Name can't be empty.")
                continue

        # Save to DB (insert if new, keep existing if email found)
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()
        cursor.execute("SELECT id, name FROM user_profile WHERE email = ?", (email,))
        row = cursor.fetchone()

        if row is None:
            cursor.execute(
                "INSERT INTO user_profile (name, email, created_at) VALUES (?, ?, ?)",
                (name, email, datetime.utcnow().isoformat())
            )
            conn.commit()
            user_id = cursor.lastrowid
        else:
            user_id, existing_name = row
            # Keep the existing name and ignore the newly typed one
            name = existing_name

        conn.close()
        return {"user_id": user_id, "name": name, "email": email}



# --- 1. DATABASE SETUP ---

def setup_database():
    """
    Connects to the existing database to ensure it is accessible and contains the required tables.
    This function no longer creates tables or populates data.
    """
    try:
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()

        # Check if the required tables exist to provide a helpful error message.
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='faq_db'")
        if cursor.fetchone() is None:
            print(f"Error: Table 'faq_db' not found in {DB_FILE}.")
            print("Please ensure your database has the correct table schema.")
            exit()
        
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='faq_db_pattern'")
        if cursor.fetchone() is None:
            print(f"Error: Table 'faq_db_pattern' not found in {DB_FILE}.")
            print("Please ensure your database has the correct table schema.")
            exit()
            
        conn.close()
        print("Database connection successful. Using existing data.")
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        exit()


# --- 2. NLP & INTENT RECOGNITION ---

def preprocess_text(text):
    """
    Processes user input by converting to lowercase, removing punctuation,
    and lemmatizing tokens.
    
    Args:
        text (str): The raw user input.
        
    Returns:
        list: A list of lemmatized tokens.
    """
    doc = nlp(text.lower())
    lemmas = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]
    return lemmas

def get_intent(user_input):
    """
    Determines the user's intent by matching their preprocessed input against
    patterns from the database. It calculates a score for each potential intent
    and returns the one with the highest score.

    Args:
        user_input (str): The raw text from the user.

    Returns:
        tuple: A tuple containing the best intent (str) and any extracted entity (like an order number).
    """

    lemmas = preprocess_text(user_input)
    lemmatized_input_str = " ".join(lemmas)

    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("SELECT intent, type, pattern, weight FROM faq_db_pattern")
    all_patterns = cursor.fetchall()
    conn.close()

    intent_scores = {}
    extracted_entity = None

    for intent, type, pattern, weight in all_patterns:
        if intent not in intent_scores:
            intent_scores[intent] = 0.0

        if type == 'keyword':
            # For keywords, we also lemmatize the pattern for a fair comparison
            pattern_lemmas = preprocess_text(pattern)
            # Check if all words in the pattern are in the user's input
            if all(p_lemma in lemmas for p_lemma in pattern_lemmas):
                intent_scores[intent] += weight
                # print(f"[MATCH] intent={intent}, type=keyword, pattern='{pattern}', +{weight} → total={intent_scores[intent]}")
        
        elif type == 'regex':
            # For regex, we match against the original (lowercased) input
            match = re.search(pattern, user_input.lower())
            if match:
                intent_scores[intent] += weight
                # print(f"[MATCH] intent={intent}, type=regex, pattern='{pattern}', +{weight} → total={intent_scores[intent]}")
                # If the regex has a capturing group, we extract it as an entity
                if match.groups():
                    extracted_entity = match.group(1)
                    # print(f"   [ENTITY] extracted → {extracted_entity}")
    
    # Show all final scores before picking
    # print("\n--- Final intent scores ---")
    # for intent, score in intent_scores.items():
    #     print(f"{intent}: {score}")


    # Determine the best intent
    if not any(score > 0 for score in intent_scores.values()):
        best_intent = 'fallback'
    else:
        best_intent = max(intent_scores, key=intent_scores.get)

    return best_intent, extracted_entity


def get_answer_for_intent(intent):
    """
    Retrieves the corresponding answer for a given intent from the database.

    Args:
        intent (str): The intent to find an answer for.

    Returns:
        str: The answer text.
    """
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("SELECT answer FROM faq_db WHERE intent = ?", (intent,))
    result = cursor.fetchone()
    conn.close()
    
    # It's good practice to have a default fallback answer in your database
    # but this handles cases where an intent might not have a matching answer.
    if result is None:
        # Plain safe fallback (do NOT call fallback_menu here)
        return "\nSorry, I couldn't understand your request. Please choose an option:"
    return result[0]


def fallback_menu():
    """
    Presents a menu when fallback is triggered.
    Returns the selected intent slug, or None after 3 invalid tries.
    """
    menu = [
        ("1) Order Status", "track_order"),
        ("2) Create Your Account", "create_account"),
        ("3) Return & Refund", "return_policy"),
        ("4) Product Damage", "package_lost_damaged"),
        ("5) Contact Us", "contact_customer_support"),
        ("6) Other request", "send_glink"),
    ]

    # print("\nSorry, I couldn't understand your request. Please choose an option:")
    for i, (label, _) in enumerate(menu, 1):
        print(f"{i}. {label}")

    attempts = 0
    while attempts < 3:
        choice = input("Enter the number of your choice (1-5): ").strip()
        try:
            idx = int(choice)
            if 1 <= idx <= len(menu):
                return menu[idx - 1][1]   # return the intent slug
        except ValueError:
            pass
        attempts += 1
        print("Invalid input. Please enter a number between 1 and 5.")

    # After 3 invalid attempts
    print("It seems you're having trouble. Please fill out this form and our team will assist you: #")
    return None

def confirm_before_end(conversation_context):
    """
    Ask user if they want to continue before ending the session.
    Returns: (updated_context, should_end: bool)
    """
    while True:
        reply = input("Bot: Is there anything else I can help you with before ending the session? (yes/no) ").strip().lower()
        if reply in ("y", "yes"):
            # reset flags but keep user for name-echo
            bot_send("Sure—how else can I help you?", min_delay=0.8, max_delay=1.2)
            return {'user': conversation_context.get('user')}, False
        elif reply in ("n", "no"):
            bot_send("Goodbye!", min_delay=0.8, max_delay=1.2)
            return conversation_context, True
        else:
            print("Bot: Please answer yes or no.")



# --- 3. DECISION TREE (CONVERSATIONAL FLOW) & MAIN LOOP ---

def chatbot_response(user_input, conversation_context):
    """
    Main decision-making hub for the chatbot.
    Handles multi-turn flows, fallback menu routing, and preserves user info in context.
    """

    def _preserve_user(ctx):
        """Keep user info so your "[NAME SAID]" echo won't break."""
        return {'user': ctx.get('user')}

    # 1) If we're waiting for an order number, try to capture it. This state check is crucial and remains.
    # if conversation_context.get('waiting_for') == 'order_number':
    #     order_number = re.search(r'\d+', user_input)
    #     if order_number:
    #         response = f"Thank you! I am now checking the status for order #{order_number.group(0)}. Please give me a moment."
    #         conversation_context = _preserve_user(conversation_context)  # Clear flags but keep user
    #     else:
    #         response = "That doesn't look like a valid order number. Please provide the numeric order ID."
    #         # Keep waiting_for as-is so the next input is still treated as an order number
    #     return response, conversation_context

    # 2) Standard intent recognition
    intent, entity = get_intent(user_input)

    # 3) Decision tree
    if intent == 'fallback':
        # ALL complex or unrecognized intents now flow through this single fallback block.
        print(get_answer_for_intent('fallback')) # Print the "I don't understand" message
        selected_intent = fallback_menu() # Show the user a menu of options

        # if selected_intent == 'order_status':
        #     # MOVED: The logic from the old 'order_status' block is now here.
        #     # Ask for order number and set the waiting flag.
        #     response = get_answer_for_intent('order_status')
        #     ctx = _preserve_user(conversation_context)
        #     ctx['waiting_for'] = 'order_number'
        #     conversation_context = ctx
        
        if selected_intent == 'goodbye':
            # MOVED: The logic from the old 'goodbye' block is now here.
            response = get_answer_for_intent('goodbye')
            ctx = _preserve_user(conversation_context)
            ctx['end_session'] = True
            conversation_context = ctx
        
        elif selected_intent == 'send_glink':
            response = "I am regret cant help you, please fill up the google form and our support will get in touch with you soon!"
            ctx = _preserve_user(conversation_context)
            ctx['end_session'] = True
            conversation_context = ctx
            
        elif selected_intent is None:
            # User chose to cancel or entered an invalid option from the menu.
            response = "No problem. Ending the session. Have a great day!"
            ctx = _preserve_user(conversation_context)
            ctx['end_session'] = True
            conversation_context = ctx

        else:
            # This handles any OTHER intent that might be selectable from the menu
            # but is a simple, single-turn response.
            response = get_answer_for_intent(selected_intent)
            conversation_context = _preserve_user(conversation_context)

    else:
        # This block now handles ALL other recognized, simple, single-turn intents (e.g., 'greeting').
        # The 'order_status' and 'goodbye' intents are no longer explicitly handled here.
        response = get_answer_for_intent(intent)
        conversation_context = _preserve_user(conversation_context)

    return response, conversation_context



def bot_send(response, min_delay=3, max_delay=5):
    """Simulate bot thinking/typing, then print the response."""
    delay = random.uniform(min_delay, max_delay)

    # typing indicator
    msg = "Bot is typing..."
    sys.stdout.write(msg)
    sys.stdout.flush()
    t0 = time.time()
    dot = 0
    while time.time() - t0 < delay:
        sys.stdout.write("." * ((dot % 3) + 1) + "\r" + msg + "   \r")
        sys.stdout.flush()
        time.sleep(0.4)
        dot += 1

    # clear the line and print the actual bot message
    sys.stdout.write(" " * (len(msg) + 3) + "\r")
    sys.stdout.flush()
    print(f"Bot: {response}", flush=True)


def main():

        # One-line, anchored regex so we only trigger when the whole input is an end cue.
    END_TRIGGER_RE = re.compile(
        r"^\s*(?:"
        r"quit|exit|bye|goodbye|"
        r"ok(?:ay)?|can|done|got it|"
        r"no more|nothing else|that(?:'|’)?s all|all good|i(?:'|’)?m good|im good|"
        r"no thanks|no thank you|thanks"
        r")\s*[.!?]*\s*$",
        re.IGNORECASE
    )

    """Main function to run the chatbot."""
    setup_database()
    ensure_user_table()

    # 🔹 Single-shot capture
    user = capture_user_profile()

    print("\n--- E-commerce Chatbot ---")
    print("Bot: Hello! How can I assist you today?")

    conversation_context = {"user": user}


    while True:
        user_input = input("You: ")
        print(f"You: {user_input}", flush=True)

        # 🔹 If user typed quit, confirm before ending
                # 🔹 Treat many phrases as “I’m done”
        if END_TRIGGER_RE.match(user_input):
            # Confirm before actually ending
            conversation_context, should_end = confirm_before_end(conversation_context)
            if should_end:
                break
            else:
                # user wants to continue → skip intent this turn
                continue

        # Normal turn
        response, conversation_context = chatbot_response(user_input, conversation_context)
        bot_send(response, min_delay=3, max_delay=5)

        # 🔹 If any branch set end_session, confirm before ending
        if conversation_context.get('end_session'):
            conversation_context, should_end = confirm_before_end(conversation_context)
            if should_end:
                break
            else:
                # user wants to continue → keep chatting
                continue


if __name__ == "__main__":
    main()

Database connection successful. Using existing data.

--- E-commerce Chatbot ---
Bot: Hello! How can I assist you today?
You: can I place an order use phone?
Bot: Unfortunately, we do not accept orders over the phone. Please place your order through our website for a smooth and secure transaction.
You: ok
Bot: Goodbye!      
